# Use of Mistral AI in processing restaurant orders

In [ ]:
# !pip install ctransformers[cuda] # For GPU run
# !pip install ctransformers[gptq] # other
!pip install ctransformers # For CPU run 

In [1]:
from ctransformers import AutoModelForCausalLM

c:\Program Files\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Loading the model (if it is first time, the model will be downloaded)

In [2]:
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF", model_file="mistral-7b-instruct-v0.1.Q5_K_M.gguf", model_type="mistral")

Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


#### Predicting and printing a restaurant order

In [3]:
# Using temperature 0 in order to make the model be as percise as possible (no random output)
print(llm(
    "I want to order Mac Crispy without pickle and 5 large french fries. Change 2 of french fries to 2 big macs and add a coke. \
    Add one small coke and make previous one big.\
    What did I order? Answer with in a jason format, which includes fileds: food_name, quantity, comment. Answer:", \
                max_new_tokens=2048, temperature=0.0, top_k=55, top_p=0.9, repetition_penalty=1.2))

 { "food_name": "Mac Crispy", "quantity": 1, "comment": "without pickle" }, { "food_name": "French Fries", "quantity": 3, "comment": "" }, { "food_name": "Big Macs", "quantity": 2, "comment": "" }, { "food_name": "Coke", "quantity": 1, "comment": "big" }


#### Most successfull experiments with prompts

In [7]:
order = """
    I would like to order Mac Crispy without pickle and 5 large french fries. Change 2 of french 
    fries to 2 big macs and add a coke.
    """.strip()
    
p = f"""
    Read the restaurant order delimited by triple backticks and return a JSON file depending on \
    what customer ordered with the following keys: food_id, quantity, comment. 
    
    Order: ```{order}```
    JSON:
    """.strip()
print(llm(p, max_new_tokens=2048, temperature=0.0, top_k=55, top_p=0.9, repetition_penalty=1.2))

 {
        "food_id": "Mac Crispy",
        "quantity": 1,
        "comment": "without pickle"
    },
    {
        "food_id": "French Fries",
        "quantity": 3,
        "comment": ""
    }


In [10]:
order = """
    I would like to order Mac Crispy without pickle and 5 large french fries. Change 2 of french \
    fries to 2 big macs and add a coke.
    """.strip()
    
p = f"""
    Extract relevant information from a restaurant order in JSON format including information like: \
    food_id (name), comment and quantity of food. Return a JSON file with the following keys: 'food_id', \
    'quantity`, `comment.
    When you will see the order think slowly what was ordered, step by step analyze what food items were added \
    and if any were removed/changed. After you analyze all food that  was ordered, its portion size and write down in JSON.
    
    Order: ```{order}```
    JSON:
    """.strip()
pred = llm(p, max_new_tokens=2048, temperature=0.0, top_k=55, top_p=0.9, repetition_penalty=1.2)
print(pred)


In [17]:
order = """
    I would like to order Mac Crispy without pickle and 5 large french fries. Change 2 of french \
    fries to 2 big macs and add a coke. Actually change one big mac to coke.
    """.strip()
    
p = f"""
    Extract relevant information from a restaurant order (delimited by triple backticks) in JSON format including information like: \
    dish name, comment and quantity of this dish. Return a JSON file with the following keys: 'dish', \
    'quantity`, `comment. Structure can look the following way:
    "menu": [
        "food_name": something
        "quantity": something integer
        "comment": something
    ]
    When you will see the order think slowly what was ordered, step by step analyze what food items were added \
    and if any were removed/changed. After you analyze all food that  was ordered, its portion size and write down in JSON.
    
    Order: ```{order}```
    JSON:
    """.strip()
pred = llm(p, max_new_tokens=2048, temperature=0.0, top_k=55, top_p=0.9, repetition_penalty=1.2)
print(pred)



        {
            "menu": [
                {
                    "dish": "Mac Crispy",
                    "quantity": 1,
                    "comment": ""
                },
                {
                    "dish": "French Fries",
                    "quantity": 3,
                    "comment": ""
                },
                {
                    "dish": "Coke",
                    "quantity": 1,
                    "comment": ""
                }
            ]
        }
